## Structural Equation Test to the Bell DAG:

given Bell scenario correlations
$P(A,B,X,Y) =  \sum_\lambda P(A| X, \lambda) P(B|Y, \lambda) P(X)P(Y)P(\lambda) \qquad \forall \lambda \in [0,card]$  

In [1]:
import gurobipy as gp
from gurobipy import GRB, norm
import itertools
import numpy as np

In [2]:
card_A = 2
card_B = 2
card_X = 2
card_Y = 2


# P_AB_giv_XY_dict = {(0, 0, 0, 0): 0.210165678052772,
#  (0, 0, 0, 1): 0.212108815708734,
#  (0, 0, 1, 0): 0.272550623849458,
#  (0, 0, 1, 1): 0.214358764573532,
#  (0, 1, 0, 0): 0.284107179382287,
#  (0, 1, 0, 1): 0.248721619963183,
#  (0, 1, 1, 0): 0.221722233585600,
#  (0, 1, 1, 1): 0.246471671098384,
#  (1, 0, 0, 0): 0.240949069339333,
#  (1, 0, 0, 1): 0.239005931683371,
#  (1, 0, 1, 0): 0.219165473511966,
#  (1, 0, 1, 1): 0.277357332787891,
#  (1, 1, 0, 0): 0.264778073225608,
#  (1, 1, 0, 1): 0.300163632644713,
#  (1, 1, 1, 0): 0.286561669052976,
#  (1, 1, 1, 1): 0.261812231540192}


# P_AB_giv_XY_dict = {(0, 0, 0, 0): 0.434,
#  (0, 0, 0, 1): 0.034,
#  (0, 0, 1, 0): 0.099,
#  (0, 0, 1, 1): 0.206,
#  (0, 1, 0, 0): 0.419,
#  (0, 1, 0, 1): 0.21,
#  (0, 1, 1, 0): 0.288,
#  (0, 1, 1, 1): 0.48,
#  (1, 0, 0, 0): 0.248,
#  (1, 0, 0, 1): 0.049,
#  (1, 0, 1, 0): 0.278,
#  (1, 0, 1, 1): 0.06,
#  (1, 1, 0, 0): 0.469,
#  (1, 1, 0, 1): 0.114,
#  (1, 1, 1, 0): 0.14,
#  (1, 1, 1, 1): 0.472}

P_AB_giv_XY_dict = {(0, 0, 0, 0): 1/2,
    (0, 0, 0, 1): 1/4,
    (0, 0, 1, 0): 1/4,
    (0, 0, 1, 1): 1/2,
    (0, 1, 0, 0): 0,
    (0, 1, 0, 1): 1/4,
    (0, 1, 1, 0): 1/4,
    (0, 1, 1, 1): 0,
    (1, 0, 0, 0): 0,
    (1, 0, 0, 1): 1/4,
    (1, 0, 1, 0): 1/4,
    (1, 0, 1, 1): 0,
    (1, 1, 0, 0): 1/2,
    (1, 1, 0, 1): 1/4,
    (1, 1, 1, 0): 1/4,
    (1, 1, 1, 1): 1/2}

In [3]:
# Define the four functions
## Bin(INT) -> Bin(INT)
def identity(x):
    return x
def swap(x):
    return 1-x
def to0(_):
    return 0
def to1(_):
    return 1

basic_strategies = [identity, swap, to0, to1]
all_strategies = list(itertools.product(basic_strategies, repeat=2))

def generate_top(u):
    f_u, g_u = all_strategies[u]
    # value 1 at point(a,b) = (f_u(x),g_u(y)), zero everywhere else
    def middle_entry(x,y):
        # value 1 at (f_u(x),g_u(y)) zero everywhere else
        middle = np.zeros((2,2), dtype=int)
        middle[f_u(x), g_u(y)] = 1
        return middle
    top_level = np.array([
                [middle_entry(0,0), middle_entry(1,0)], 
                [middle_entry(0,1), middle_entry(1,1)]
                ]) 
    return top_level



# declaring Gurobi variable W_i w/ i in {0,1,...,15}, and declaring v
m1 = gp.Model("m1")
m1.setParam('OutputFlag', 0)
W = m1.addVars(16, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="W")
m1.update()
global is_compatible

internal_gurobi_expr = sum([W[i]*generate_top(i) for i in range(16)])
m1.addConstr(W.sum() == 1, "sum(W_i) == 1")
for x,y, a,b in itertools.product(range(2), repeat=4):
    m1.addConstr(internal_gurobi_expr[x,y,a,b] == P_AB_giv_XY_dict[a,b,x,y], f"P({a},{b}|{x},{y}) == {internal_gurobi_expr[x,y,a,b]} == {P_AB_giv_XY_dict[(a,b,x,y)]}")
m1.update()
m1.optimize()

if m1.status == 2: # GRB.OPTIMAL
    is_compatible = True
else: 
    is_compatible = False

print("Compatibility:", m1.status, is_compatible)
if not is_compatible:
    raise SystemExit("DAG not compatible with given dist")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Compatibility: 2 True


In [4]:
def asses_compat_with_card(P_AB_giv_XY_numeric: np.ndarray, card_l: int, verbose=0) -> bool:
    m = gp.Model("m")
    m.reset()
    m.setParam('OutputFlag', verbose)
    m.params.NonConvex = 2 # Using quadratic equality constraints.


    # variables
    P_l = m.addMVar(card_l, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="P_l")
    P_A_giv_X_l = m.addMVar(shape = (card_A, card_X, card_l), vtype=GRB.CONTINUOUS, name="P(A|X,l)", lb=0, ub=1)
    P_B_giv_Y_l = m.addMVar(shape = (card_B, card_Y, card_l), vtype=GRB.CONTINUOUS, name="P(B|Y,l)", lb=0, ub=1)
    P_BL_giv_Y = m.addMVar(shape = (card_B, card_Y, card_l), vtype=GRB.CONTINUOUS, name="P(B,l|Y)", lb=0, ub=1)
    m.update()

    for b,y,l in np.ndindex(card_B, card_Y, card_l):
        m.addConstr(P_BL_giv_Y[b,y,l] == P_B_giv_Y_l[b,y,l] * P_l[l], f"prod[{b},{y},{l}] = P(B|Y,l) * P(l)")

    # structural eqn
    for a,b,x,y in np.ndindex(card_A, card_B, card_X, card_Y):
        m.addConstr(P_AB_giv_XY_numeric[a,b,x,y] == sum([P_A_giv_X_l[a,x,l] * P_BL_giv_Y[b,y,l] for l in range(card_l)]), f"eqn[{a},{b},{x},{y}]")




    m.addConstr(sum([P_l[l] for l in range(card_l)]) == 1, "sum_P_l = 1")
    for x, l in np.ndindex(card_X, card_l):
        m.addConstr(P_A_giv_X_l[0,x,l] + P_A_giv_X_l[1,x,l] == 1, f'P(|{x,l}) = 1')
    for y, l in np.ndindex(card_Y, card_l):
        m.addConstr(P_B_giv_Y_l[0,y,l] + P_B_giv_Y_l[1,y,l] == 1, f'P(|{y,l}) = 1')

    m.optimize()
    if m.status == 2: # GRB.OPTIMAL
        return True
    else: 
        return False


In [5]:
for card_l in range(9):
    model_feasibility = asses_compat_with_card(P_AB_giv_XY_dict, card_l, verbose=0)
    if model_feasibility == True:
        print("Bell DAG compatible with card_l:", card_l)
        break

# For dstribution:
P_AB_giv_XY_dict

Discarded solution information
Discarded solution information
Discarded solution information
Discarded solution information
Discarded solution information
Bell DAG compatible with card_l: 4


{(0, 0, 0, 0): 0.5,
 (0, 0, 0, 1): 0.25,
 (0, 0, 1, 0): 0.25,
 (0, 0, 1, 1): 0.5,
 (0, 1, 0, 0): 0,
 (0, 1, 0, 1): 0.25,
 (0, 1, 1, 0): 0.25,
 (0, 1, 1, 1): 0,
 (1, 0, 0, 0): 0,
 (1, 0, 0, 1): 0.25,
 (1, 0, 1, 0): 0.25,
 (1, 0, 1, 1): 0,
 (1, 1, 0, 0): 0.5,
 (1, 1, 0, 1): 0.25,
 (1, 1, 1, 0): 0.25,
 (1, 1, 1, 1): 0.5}

In [6]:
# card_l = 4 # hidden common cause

# m = gp.Model("m")
# m.reset()
# m.setParam('OutputFlag', 0)
# m.params.NonConvex = 2 # Using quadratic equality constraints.

# # variables
# P_l = m.addMVar(card_l, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="P_l")
# P_A_giv_X_l = m.addMVar(shape = (card_A, card_X, card_l), vtype=GRB.CONTINUOUS, name="P(A|X,l)", lb=0, ub=1)
# P_B_giv_Y_l = m.addMVar(shape = (card_B, card_Y, card_l), vtype=GRB.CONTINUOUS, name="P(B|Y,l)", lb=0, ub=1)
# prod = m.addMVar(shape = (card_A, card_Y, card_l), vtype=GRB.CONTINUOUS, name="prod", lb=0, ub=1)
# m.update()


# for a, b, x, y in np.ndindex(card_A, card_B, card_X, card_Y):
#     P_AB_giv_XY_val = P_AB_giv_XY[(a, b, x, y)]
#     RHS_obs = gp.QuadExpr()
#     for l in range(card_l):
#         m.addConstr(prod[a,x,l] == P_l[l] * P_A_giv_X_l[a,x,l])
#         RHS_obs += prod[a,x,l]*P_B_giv_Y_l[b, y, l]
#     m.addConstr(P_AB_giv_XY_val == RHS_obs)


# for l in range(card_l):
#     m.addConstr(gp.quicksum(P_l[l] for l in range(card_l)) == 1, "sum_P_l = 1")
#     # m.addConstr(gp.quicksum(P_A_giv_X_l[a, x, l] for a, x in np.ndindex(card_A, card_X)) == 1, f"sum_P_A_giv_X_l_{l} = 1")
#     m.addConstr(gp.quicksum(P_B_giv_Y_l[b, 0, l] for b in [0, 1]) == 1, f"sum_P_B_giv_Y_l_{l} = 1")
#     # m.addConstr(gp.quicksum(P_B_giv_Y_l[b, 1, l] for b in [0, 1]) == 1, f"sum_P_B_giv_Y_l_{l} = 1")
#     m.addConstr(gp.quicksum(P_A_giv_X_l[a, 0, l] for a in [0, 1]) == 1, f"sum_P_A_giv_X_l{l} = 1")
#     m.addConstr(gp.quicksum(P_A_giv_X_l[a, 1, l] for a in [0, 1]) == 1, f"sum_P_A_giv_X_l{l} = 1")

# m.optimize()
# m.status

In [7]:
# def P_A(a) from P_AB_giv_XY:
# def P_A(a):
#     return sum([P_AB_giv_XY[a,b,x,y] for b,x,y in itertools.product(range(card_B), range(card_X), range(card_Y))])
# def P_B(b):
#     return sum([P_AB_giv_XY[a,b,x,y] for a,x,y in itertools.product(range(card_A), range(card_X), range(card_Y))])
# for a in range(card_A):
#     m.addConstr(P_A_giv_X_l[a,:,:].sum() == P_A(a), f"sum_P_A_giv_X_l[{a}] = P_A({a})")
# for b in range(card_B):
#     m.addConstr(P_B_giv_Y_l[b,:,:].sum() == P_B(b), f"sum_P_B_giv_Y_l[{b}] = P_B({b})")

In [8]:
# with gp.Env(empty=True) as env:
#     env.setParam('OutputFlag', 0)
#     env.start()
#     m = gp.Model(env=env)
#     card_l = 0 # hidden common cause

#     while True:
#         card_l += 1
#         print('Trying card_l =', card_l)
#         m.reset()
#         m.params.NonConvex = 2  # Using quadratic equality constraints.


#         # variables
#         P_l = m.addMVar(card_l, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="P_l")
#         P_A_giv_X_l = m.addMVar(shape = (card_A, card_X, card_l), vtype=GRB.CONTINUOUS, name="P(A|X,l)", lb=0, ub=1)
#         P_B_giv_Y_l = m.addMVar(shape = (card_B, card_Y, card_l), vtype=GRB.CONTINUOUS, name="P(B|Y,l)", lb=0, ub=1)
#         prod = m.addMVar(shape = (card_A, card_B, card_X, card_Y, card_l), vtype=GRB.CONTINUOUS, name="quad_prod", lb=0, ub=1) # P(A|X,l) * P(B|Y,l)
#         m.update()

#         def P_Y(Y):
#             return sum([dist_ABXY[(a,b,x,Y)] for a,b,x in np.ndindex(card_A, card_B, card_X)])
#         def P_X(X):
#             return sum([dist_ABXY[(a,b,X,y)] for a,b,y in np.ndindex(card_A, card_B, card_Y)])
#         def P_XYl(x,y,l):
#             return P_X(x) * P_Y(y) * P_l[l]

#         for a,b,x,y, l in np.ndindex(card_A, card_B, card_X, card_Y, card_l):
#             m.addConstr(prod[a,b,x,y,l] == P_A_giv_X_l[a,x,l] * P_B_giv_Y_l[b,y,l])

#         # structural eqn
#         for a,b,x,y in np.ndindex(card_A, card_B, card_X, card_Y):
#             dist_ABXY[a,b,x,y] == sum([prod[a,b,x,y,l] * P_XYl(x,y,l) for l in range(card_l)])

#         # sums to 1
#         m.addConstr(sum([P_l[l] for l in range(card_l)]) == 1, "sum_P_l = 1")
#         m.addConstr(P_A_giv_X_l[0,:,:]+P_A_giv_X_l[1,:,:] == 1 , "sum_P_A_giv_X_l = 1")
#         m.addConstr(P_B_giv_Y_l[0,:,:]+P_B_giv_Y_l[1,:,:] == 1 , "sum_P_B_giv_Y_l = 1")

#         m.optimize()
#         if m.status == 2: # GRB.OPTIMAL
#             print('Optimal solution found with card_l =', card_l)
#             break
#         else:
#             print("Optimization failed with card_l =", card_l)